### Main Libraries Needed

In [1]:
#Import of main libraries

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np 
from __future__ import division, print_function
from pandas import read_csv
from pandas import read_excel
from pandas import DataFrame
from pandas import ExcelWriter
from pandas import ExcelFile

#Import of the pyomo module
from pyomo.environ import *

### Importing and creating necessary Data Frames

In [2]:
#Import Bids, Lanes, and Carrier DF's
bidsDf = read_csv("Benelux Bids.csv")
bidsDf = bidsDf.drop("Unnamed: 0", axis = 1)
lanesDf = read_csv("Benelux Lanes.csv")
lanesDf = lanesDf.drop("Unnamed: 0", axis = 1)
CarriersDf = read_csv("Carriers.csv")
CarriersDf = CarriersDf.drop("Unnamed: 0", axis = 1)

In [3]:
#Create data frames for delta matrix
deltaDf = DataFrame(np.zeros((len(lanesDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    deltaDf.at[bidsDf.loc[bid,'Cluster Lot Index'] - 1, bid] = 1

In [4]:
# Create the eta DF
etaDf = DataFrame(np.zeros((len(CarriersDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    etaDf.at[bidsDf.loc[bid,'Carrier Index'] - 1, bid] = 1

In [5]:
import pandas as pd

In [6]:
Mc = pd.Series(np.sum(etaDf, axis = 1))

In [7]:
Mc

0     126.0
1     175.0
2     144.0
3     174.0
4      76.0
5     142.0
6      30.0
7     135.0
8     158.0
9     122.0
10     22.0
11     86.0
12    144.0
13     77.0
14      1.0
15    109.0
dtype: float64

### Create the necessary dictionaries

In [8]:
carriers = Mc.to_dict()

bidValues = dict()
for bid in bidsDf.index:
    bidValues[bid] = bidsDf.loc[bid, 'Cost']

demandValues = dict()
for lane in lanesDf.index:
    demandValues[lane] = 1

delta = dict()
for lane in lanesDf.index:
    for bid in bidsDf.index:
        delta[(lane,bid)] = deltaDf.loc[lane,bid]

eta = dict()
for c in CarriersDf.index:
    for bid in bidsDf.index:
        eta[(c,bid)] = etaDf.loc[c,bid]


In [9]:
# Create the Concrete Model
model = ConcreteModel()

model.numBids = len(bidsDf.index)

model.numItems = len(lanesDf.index)

model.numCarriers = len(CarriersDf.index)

model.BIDS = Set(initialize = bidsDf.index.values)

model.LANES = Set(initialize = lanesDf.index.values)

model.CARRIERS = Set( initialize = CarriersDf.index.values)

model.M = Param(model.CARRIERS, initialize = carriers)

model.bidValue = Param(model.BIDS, initialize = bidValues, doc='Value of each bid in the program')

model.demand = Param(model.LANES, initialize = demandValues, doc='Total demand on each lane')

# Below is where I made the suggested work around from https://stackoverflow.com/questions/45616967/pyomo-valueerror-invalid-constraint-expression

model.delta = Param(model.LANES, model.BIDS, initialize= delta, doc='delta gives information regarding which lanes are in a bid package')#,mutable = True)

model.eta = Param(model.CARRIERS, model.BIDS, initialize= eta, doc='eta gives information regarding which carriers are in a bid package', mutable = True)

model.x = Var(model.BIDS, domain = Binary, doc='Decision variable for each bid in the program')

model.z = Var(model.CARRIERS, domain = Binary, doc='Decision variable for each carrier in the program')

model.maxCarriers = 5
model.minCarriers = 1

### Set Model Obj and Constraints

In [10]:
def obj_expression(model):
    return sum(model.bidValue[i]*model.x[i] for i in model.BIDS)
model.OBJ = Objective(rule=obj_expression, sense=minimize, doc='Objective function definition')

In [12]:
def constraint_rule(model, l):
    return sum(model.delta[l,b]*model.x[b] for b in model.BIDS) <= model.demand[l]
model.xConstraint = Constraint(model.LANES, rule=constraint_rule)

def demand_constraint_rule(model):
    return sum(model.x[b] for b in model.BIDS) >= model.numItems
model.demandConstraint = Constraint(rule=demand_constraint_rule)

def constraint2_rule(model, k):
    return sum(model.eta[k,b]*model.x[b] for b in model.BIDS) - model.M[k]*model.z[k]<=0
model.upperBoundConstraint = Constraint(model.CARRIERS, rule=constraint2_rule)

def constraint3_rule(model, k):
    return model.z[k] - sum(model.eta[k,b]*model.x[b] for b in model.BIDS)<=0
model.lowerBoundConstraint = Constraint(model.CARRIERS, rule=constraint3_rule)

def constraint4_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)<=model.maxCarriers
model.zConstraint = Constraint(rule=constraint4_rule)

def constraint5_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)>=model.minCarriers
model.zConstraint2 = Constraint(rule=constraint5_rule)

    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


    (type=<class 'pyomo.core.base.constraint.SimpleConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.SimpleConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


### Run the Model

In [13]:
def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()

In [14]:
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("glpk")
%timeit results = opt.solve(model)

1.8 s ± 77.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
results = opt.solve(model)
model.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 466363.2
  Upper bound: 466363.2
  Number of objectives: 1
  Number of constraints: 216
  Number of variables: 1738
  Number of nonzeros: 6949
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 663
      Number of created subproblems: 663
  Error rc: 0
  Time: 0.9226522445678711
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    OBJ:
      Value: 466363.19999999995
  Variable:
    x[0]:
      Value: 1
    x[1002]:
      Value: 1
    x[1010]:
      Value: 1
    x[1021]:
      Value: 1
    x[1028]:
      Value: 1
    x[1041]:
      Value: 1
    x[1052]:
      Value: 1
    x[105]:
      Value: 1
    x[1063]:
      Value: 1
    x[1077]:
      Value: 1
    x[1089]:
      Value: 1
    x[1103]:
      Value: 1
    x[1111]:
      Value: 1
    x[1121]:
      Value: 1
    x[1130]

In [16]:
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)

In [0]:
#model.pprint()

## Store results


In [17]:
winningBids = []
index = 0
bidNum = 0
for bids in range(len(bidsDf.index)):
    if model.x[bids].value > 0:
        winningBids.append(bidNum)
    bidNum += 1
    index += 1

winningBidsDf = bidsDf.iloc[winningBids]
winningBidsDf.head()

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,INTT@ISO_001,INTT,ISO_001,1,2864.4,1,2864.4,0.0,3,Benelux,6,1
11,INTT@ISO_002,INTT,ISO_002,2,2337.6,1,2337.6,0.0,3,Benelux,6,2
22,INTT@ISO_003,INTT,ISO_003,3,2456.4,1,2456.4,0.0,2,Benelux,6,3
30,ITLW@ISO_004,ITLW,ISO_004,4,2214.0,1,2214.0,0.0,1,Benelux,5,4
36,ITLW@ISO_005,ITLW,ISO_005,5,2378.4,2,2143.2,235.2,6,Benelux,5,5


In [17]:
winningBidsDf

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,INTT@ISO_001,INTT,ISO_001,1,2864.4,1,2864.4,0.0,3,Benelux,6,1
11,INTT@ISO_002,INTT,ISO_002,2,2337.6,1,2337.6,0.0,3,Benelux,6,2
22,INTT@ISO_003,INTT,ISO_003,3,2456.4,1,2456.4,0.0,2,Benelux,6,3
30,ITLW@ISO_004,ITLW,ISO_004,4,2214.0,1,2214.0,0.0,1,Benelux,5,4
36,ITLW@ISO_005,ITLW,ISO_005,5,2378.4,2,2143.2,235.2,6,Benelux,5,5
47,BLKH@ISO_006,BLKH,ISO_006,6,2718.0,1,2718.0,0.0,14,Benelux,2,6
59,ITLW@ISO_007,ITLW,ISO_007,7,2378.4,1,2378.4,0.0,24,Benelux,5,7
71,INTT@ISO_008,INTT,ISO_008,8,2589.6,1,2589.6,0.0,1,Benelux,6,8
84,BLKH@ISO_009,BLKH,ISO_009,9,2220.0,2,2100.0,120.0,23,Benelux,2,9
96,BLKH@ISO_010,BLKH,ISO_010,10,2322.0,3,2040.0,282.0,1,Benelux,2,10


In [18]:
Total = sum(winningBidsDf['Cost']*winningBidsDf['Shipments per Lot'])
Total

7136935.200000002

In [18]:
winningBidsDf.to_csv('Benelux Carrier Constraint WinningBids.csv')